In [1]:
from data import data
import pandas as pd
import numpy as np
from analyze import analyze
from plots import scatter, trend, lmh, trend_years, comparison_filter
from sklearn import linear_model, preprocessing
import matplotlib.pyplot as plt

In [2]:
analyze(data, 2017).drop("World", 0)

,GDP per capita (current US$),GDP per capita growth (annual %),"Agriculture, forestry, and fishing, value added (% of GDP)",Food production index (2004-2006 = 100),Arable land (hectares per person),Surface area (sq. km),Rural population (% of total population),Access to electricity (% of population),CO2 emissions (metric tons per capita),Urban population,...,Real interest rate (%),"Stocks traded, total value (% of GDP)","Listed domestic companies, total",Domestic credit to private sector (% of GDP),Bank capital to assets ratio (%),"Official exchange rate (LCU per US$, period average)",Time required to start a business (days),New businesses registered (number),Total tax rate (% of commercial profits),Economic freedom
Country Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,585.850064,0.071666,NaN,NaN,NaN,652860.0,74.750,NaN,NaN,8971345.0,...,NaN,NaN,NaN,3.474263,11.554978,68.026904,7.5,NaN,71.4,51.3
Albania,4537.862492,3.936913,18.961950,NaN,NaN,28750.0,40.617,NaN,NaN,1706345.0,...,NaN,NaN,NaN,35.003810,10.167439,119.099667,5.0,NaN,37.3,64.5
Algeria,4123.389937,-0.052730,12.264896,NaN,NaN,2381740.0,27.948,NaN,NaN,29770548.0,...,1.123428,NaN,NaN,24.363741,NaN,110.973017,20.0,NaN,65.6,44.7
American Samoa,NaN,NaN,NaN,NaN,NaN,200.0,12.830,NaN,NaN,48502.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.5
Andorra,39146.548836,2.291464,NaN,NaN,NaN,470.0,11.850,NaN,NaN,67845.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,4170.312280,-2.561034,NaN,NaN,NaN,1246700.0,35.161,NaN,NaN,19311773.0,...,-11.683023,NaN,NaN,15.789916,NaN,165.915951,36.0,NaN,49.1,48.6
Antigua and Barbuda,15021.738183,2.274424,1.630647,NaN,NaN,440.0,75.287,NaN,NaN,25210.0,...,7.631749,NaN,NaN,44.297566,NaN,2.700000,22.0,NaN,41.9,53.6
Argentina,14401.974855,1.879661,5.607975,NaN,NaN,2780400.0,8.251,NaN,NaN,40618237.0,...,4.699532,1.034015,96.0,16.112258,12.281262,16.562707,24.0,NaN,106.0,52.3
Armenia,3936.798320,7.293324,14.937978,NaN,NaN,29740.0,36.897,NaN,NaN,1849202.0,...,11.908686,NaN,NaN,51.501786,15.707337,482.716394,4.5,NaN,18.5,68.7


In [17]:
sublog = lambda x: np.log(10*x) if x > 0 else -np.log(abs(10*x)) if x < 0 else None
log = lambda x: np.array([sublog(i) for i in x])

In [34]:
x = np.array([-4.0,-3, -2, -1, 0, 1,2,3,4])

In [35]:
x.dtype = "int64"

In [36]:
x

array([-4607182418800017408, -4609434218613702656, -4611686018427387904,
       -4616189618054758400,                    0,  4607182418800017408,
        4611686018427387904,  4613937818241073152,  4616189618054758400])

In [7]:
def scatter(data, year, indexes, plot=True):
    '''Generates scatter plot and trend line or the r squared, coefficient and number of countries (plot=False) in comparison between two indexes (list of strs) in a given year (int)'''
    
    if type(year) == int:
        df = log(np.array(analyze(data, year)[indexes].dropna()))
    else:
        df = log(np.array(analyze(data, year[0])[[indexes[0]]].join(analyze(data, year[1])[indexes[1]]).dropna()))
    x = df[:,0]
    y = df[:,1]
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    r2 = metrics.r2_score(y, p(x))
    if plot:
        plt.plot(x, p(x), color='red', linewidth=2.)
        plt.scatter(x, y, color='blue', s=50, alpha=.5)
        plt.legend(["R**2 = {:0.4}".format(r2)])
        plt.title('Tendency line log-log')
        plt.xlabel(indexes[0])
        plt.ylabel(indexes[1])
        plt.show()
        
    else:
        return r2, z[0], len(x)

In [8]:
year = [2017, 2017]
indexes = ['Economic freedom', 'GDP per capita (current US$)']
df = log(np.array(analyze(data, year[0])[[indexes[0]]].join(analyze(data, year[1])[indexes[1]]).dropna()))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from analyze import analyze 
from sklearn import linear_model, preprocessing, metrics
sublog = lambda x: np.log(x) if x > 0 else -np.log(abs(x)) if x < 0 else None
log = lambda x: np.array([sublog(i) for i in x])
    
def scatter(data, year, indexes, plot=True):
    '''Generates scatter plot and trend line or the r squared, coefficient and number of countries (plot=False) in comparison between two indexes (list of strs) in a given year (int)'''
    
    if type(year) == int:
        df = np.array(analyze(data, year)[indexes].dropna())
    else:
        df = np.array(analyze(data, year[0])[[indexes[0]]].join(analyze(data, year[1])[indexes[1]]).dropna())
    x = log(df[:,0]).reshape(-1, 1)
    y = log(df[:,1]).reshape(-1, 1)
    model = linear_model.LinearRegression().fit(x, y)
    if plot:
        plt.plot(x, model.predict(x), color='red', linewidth=2.)
        plt.scatter(x, y, color='blue', s=50, alpha=.5)
        plt.legend(["R**2 = {:0.4}".format(model.score(x, y))])
        plt.title('Tendency line log-log')
        plt.xlabel(indexes[0])
        plt.ylabel(indexes[1])
        plt.show()
        
    else:
        return model.score(x, y), model.coef_[0][0], len(x)
    
def trend(data, country, index, years):
    '''Generates the trend over the years (list of ints) of the countries (list of strs) and requested index (str)'''
    
    for i in country:
        plt.plot(analyze(data, "c-"+i)[index].loc[[str(i) for i in range(years[0], years[1]+1)]])
    plt.legend(country)
    plt.title('Tendency')
    plt.setp(plt.xticks()[1], rotation=60)
    plt.xlabel('Years')
    plt.ylabel(index)
    plt.show()
    
def lmh(data, year, index, x):
    '''Generates the x (int) smaller, middle and higher values for the indicated year (int) and index (str)'''
    
    df = analyze(data, year).drop("World", 0)[index].dropna().sort_values()
    amount = int(len(df)/2)-int(x/2)
    df.iloc[list(range(0,x))+list(range(amount, amount+x))+list(range(-x,0))].plot(kind="bar")
    plt.title('Bar plot')
    plt.xlabel('Country')
    plt.ylabel(index)
    plt.show()
    
def trend_years(data, country, indexes, years, plot=True):
    '''Generates the trend line or the r squared, coefficient and number of countries (plot=False) of the years (list of ints) of the country (str) and the requested index (str)'''
    
    if type(indexes) == str:
        df = analyze(data, "c-"+country)[indexes].loc[[str(i) for i in range(years[0], years[1]+1)]].dropna()
        x = np.array(df.index, dtype="float64")
        y = preprocessing.MinMaxScaler().fit_transform(np.array(df.values, dtype="float64").reshape(-1, 1)).reshape(-1)
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        r2 = metrics.r2_score(y, p(x))
        if plot:
            plt.plot(df.index, p(x),color='red', linewidth=2.)
            plt.plot(df.index, y)
            plt.legend(["R**2 = {:0.4}".format(r2), "Coef = {:0.4}".format(z[0])])
            plt.title('Tendency line')
            plt.setp(plt.xticks()[1], rotation=60)
            plt.xlabel('Years')
            plt.ylabel(indexes)
            plt.show()

        else:
            return r2, z[0], len(x)
    else:
        df = analyze(data, "c-"+country)[indexes].loc[[str(i) for i in range(years[0], years[1]+1)]].dropna()
        for ind in indexes:
            plt.plot(df.index, preprocessing.MinMaxScaler().fit_transform(np.array(df[ind], dtype="float64").reshape(-1, 1)))
        plt.legend(indexes)
        plt.setp(plt.xticks()[1], rotation=90)
        plt.show() 
        
def comparison_filter(data, year=2014, years=[2000,2014], country="World", index='GDP per capita (current US$)', r2=0.45, coef=0.05, num=10):
    '''Generates index graphs that match the index and criteria provided'''
    
    indcoef = trend_years(data, country, index, years, False)[1]
    for ind in analyze(data, year).columns.drop(index):
        try:
            isc = scatter(data, year, [index]+[ind], False)
            itr = trend_years(data, country, ind, years, False)[1]
            if isc[0] >= r2 and abs(itr-indcoef) <= coef and isc[2] >= num:
                print(index + " and " + ind)
                scatter(data, year, [index]+[ind])
                trend_years(data, country, [index]+[ind], years)
        except:
            pass

In [37]:
sublog = lambda x: np.log(x) if x > 0 else -np.log(abs(x)) if x < 0 else None
log = lambda x: np.array([sublog(i) for i in x])

In [48]:
subcube = lambda x: x**(1/3) if x >= 0 else -abs(x)**(1/3)
cube = lambda x: np.array([subcube(i) for i in x])

In [49]:
import numpy as np
a = np.array([1,2,3,4,5])

In [51]:
cube(a**3)

array([1., 2., 3., 4., 5.])